# Deep learning

## Laboratorio Python

### Esperimento 1: Rappresentazione grafiche di reti neurali multistrato

In questo esperimento vogliamo programmare una funzione in grado di  realizzare una rappresentazione grafica di una rete neurale. A tale scopo adotteremo la libreria Python **networkx**. 
La funzione draw_mlp riceve in ingresso la descrizione della rete multistrato in termini di numero di neuroni di ingresso, numero di strati e numero di neuroni per ogni strato e numero di neuroni di uscita.
Il risultato della funzione è il disegno del grafo della rete MLP.

In [ ]:
import matplotlib.pyplot as plt
import networkx as nx

# Funzione per disegnare una rappresentazione grafica della rete MLP
def draw_mlp(hidden_layers, input_size, output_size):
    G = nx.DiGraph()
    layer_sizes = [input_size] + list(hidden_layers) + [output_size]
    
    # Posizionamento dei nodi
    pos = {}
    n_layers = len(layer_sizes)
    v_spacing = 1
    
    # Creazione dei nodi
    for i, layer_size in enumerate(layer_sizes):
        layer_top = v_spacing * (layer_size - 1) / 2
        for j in range(layer_size):
            pos[f'{i}-{j}'] = (i, layer_top - v_spacing * j)
            G.add_node(f'{i}-{j}')
    
    # Creazione degli archi
    for i, (layer_size_a, layer_size_b) in enumerate(zip(layer_sizes[:-1], layer_sizes[1:])):
        for j in range(layer_size_a):
            for k in range(layer_size_b):
                G.add_edge(f'{i}-{j}', f'{i+1}-{k}')
    
    # Disegna il grafico
    plt.figure(figsize=(12, 8))
    nx.draw(G, pos=pos, with_labels=False, arrows=False, node_size=300, node_color="lightblue")
    
    # Etichette
    for i in range(input_size):
        pos[f'0-{i}'] = (pos[f'0-{i}'][0] - 0.1, pos[f'0-{i}'][1])
        plt.text(pos[f'0-{i}'][0], pos[f'0-{i}'][1], f'Input {i+1}', horizontalalignment='right')
    
    for i in range(output_size):
        pos[f'{n_layers-1}-{i}'] = (pos[f'{n_layers-1}-{i}'][0] + 0.1, pos[f'{n_layers-1}-{i}'][1])
        plt.text(pos[f'{n_layers-1}-{i}'][0], pos[f'{n_layers-1}-{i}'][1], f'Output {i+1}', horizontalalignment='left')
    
    plt.title("Rappresentazione Grafica della Rete MLP")
    plt.show()

Applichiamo la funzione draw_mlp() al caso di una rete con 4 ingressi 3 strati nascosti da 3, 9 e 3 neuroni rispettivamente e 1 neurone di uscita:

In [ ]:
# Parametri della rete MLP utilizzata nell'esempio
hidden_layers = (3,9, 3)  # Due strati nascosti con 10 e 5 neuroni rispettivamente
input_size = 4  # Due caratteristiche in input
output_size = 1  # Un neurone di output (classificazione binaria)

# Disegnare la rappresentazione della rete MLP
draw_mlp(hidden_layers, input_size, output_size)

### Esperimento 2: Rete MLP applicata al caso di classi concentriche {#sec-lab-rete-multistrato-classi-concentriche}

In questo esperimento vogliamo applicare una rete multistrato al problema della classificazione binaria nel caso di un dataset bidimensionale composto da due classi concentrichele 
La rete è composta da:

- **Strato di input**: Due ingressi, ciascuno corrispondente a una delle caratteristiche del dataset (x1,x2).
- **Strati nascosti**: Due strati nascosti, il primo con 10 neuroni e il secondo con 5 neuroni, che permettono alla rete di apprendere rappresentazioni più complesse dei dati grazie alla funzione di attivazione non lineare "relu" adottata. Ogni neurone in un determinato strato è connesso a tutti i neuroni dello strato successivo, consentendo il flusso delle informazioni attraverso la rete durante l'addestramento e la predizione.
- **Strato di output**: Un singolo neurone di output, utilizzato per la classificazione binaria (classe 0, class 1).

Usando la funzione introdotta nell'esempio 1 possiamo disegnare la rete MLP che vogliamo adottare per risolvere il problem di classificazione nel caso di classi concentriche.

In [ ]:
# Parametri della rete MLP utilizzata nell'esempio
hidden_layers = (10, 5)  # Due strati nascosti con 10 e 5 neuroni rispettivamente
input_size = 2  # Due caratteristiche in input
output_size = 1  # Un neurone di output (classificazione binaria)

# Disegnare la rappresentazione della rete MLP
draw_mlp(hidden_layers, input_size, output_size)

In [ ]:
# Importa le librerie necessarie
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
from sklearn.preprocessing import StandardScaler

# 1. Generazione dei dati: due classi concentriche
def generate_concentric_circles(n_samples=500, noise=0.1):
    np.random.seed(42)
    n_samples_per_class = n_samples // 2
    angles = np.random.rand(n_samples_per_class) * 2 * np.pi

    inner_radius = 1 + noise * np.random.randn(n_samples_per_class)
    outer_radius = 3 + noise * np.random.randn(n_samples_per_class)

    inner_x = np.stack([inner_radius * np.cos(angles), inner_radius * np.sin(angles)], axis=1)
    outer_x = np.stack([outer_radius * np.cos(angles), outer_radius * np.sin(angles)], axis=1)

    X = np.concatenate([inner_x, outer_x], axis=0)
    y = np.array([0] * n_samples_per_class + [1] * n_samples_per_class)

    return X, y

X, y = generate_concentric_circles()

# 2. Visualizzazione dei dati
plt.figure(figsize=(6, 6))
plt.scatter(X[y == 0][:, 0], X[y == 0][:, 1], c='blue', label='Classe 0')
plt.scatter(X[y == 1][:, 0], X[y == 1][:, 1], c='green', label='Classe 1')
plt.title('Dati con Classi Concentriche')
plt.legend()
plt.grid(True)
plt.show()

# 3. Divisione del dataset e normalizzazione
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# 4. Creazione e addestramento del modello MLP
model = MLPClassifier(hidden_layer_sizes=(10, 5), activation='relu', max_iter=1000, random_state=42)
model.fit(X_train_scaled, y_train)

# 5. Valutazione: matrice di confusione
y_pred = model.predict(X_test_scaled)
cm = confusion_matrix(y_test, y_pred)
disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=["Classe 0", "Classe 1"])
disp.plot(cmap=plt.cm.Blues)
plt.title("Matrice di Confusione")
plt.show()

# 6. Grafico della superficie di decisione
h = .02  # step size
x_min, x_max = X[:, 0].min() - .5, X[:, 0].max() + .5
y_min, y_max = X[:, 1].min() - .5, X[:, 1].max() + .5
xx, yy = np.meshgrid(np.arange(x_min, x_max, h),
                     np.arange(y_min, y_max, h))

grid = np.c_[xx.ravel(), yy.ravel()]
grid_scaled = scaler.transform(grid)
Z = model.predict(grid_scaled)
Z = Z.reshape(xx.shape)

plt.figure(figsize=(8, 6))
plt.contourf(xx, yy, Z, cmap=plt.cm.Pastel2, alpha=0.8)
plt.scatter(X[y == 0][:, 0], X[y == 0][:, 1], c='blue', label='Classe 0', edgecolors='k')
plt.scatter(X[y == 1][:, 0], X[y == 1][:, 1], c='green', label='Classe 1', edgecolors='k')
plt.title("Superficie di Decisione del Modello MLP")
plt.legend()
plt.grid(True)
plt.show()

Il codice Python scritto per questo esperimento segue la seguente logica:

- **Generazione dei dati**: Due insiemi di punti sono distribuiti in cerchi concentrici: il primo (classe 0) vicino all'origine, il secondo (classe 1) su un raggio maggiore. La forma dei dati rende il problema non linearmente separabile.

- **Visualizzazione**: Il primo grafico mostra chiaramente la distribuzione circolare dei due insiemi di punti.

- **Preprocessing**: I dati vengono suddivisi in un training e test set (70% - 30%) e normalizzati con StandardScaler.

- **Modello MLP**: È stata creata una rete con due strati nascosti (10 e 5 neuroni) e funzione di attivazione ReLU. La rete viene addestrata per classificare i dati.

- **Valutazione**: Viene calcolata e mostrata la matrice di confusione, che evidenzia l'accuratezza del modello nel distinguere le due classi.

- S**uperficie di decisione**: Il terzo grafico mostra come la rete ha appreso a separare le due classi: la forma curva della regione di decisione indica che il modello ha effettivamente appreso la complessità dei dati, superando i limiti del percettrone semplice (che può solo separare linearmente).

### Esperimento 3: Rete MLP per la predizione dell'esito di un caso giudiziario

Applicazione di una rete neurale multistrato (MLP) per la predizione dell'esito di un caso giudiziario basandosi su tre caratteristiche: complessità del caso, esperienza dell'avvocato, e importanza mediatica.
La rete è composta da:

- **Strato di input**: Tre ingressi, ciascuno corrispondente a una delle caratteristiche del dataset (complessità del caso, esperienza dell'avvocato, importanza mediatica).
- **Strati nascosti**: Due strati nascosti, il primo con 10 neuroni e il secondo con 5 neuroni, che permettono alla rete di apprendere rappresentazioni più complesse dei dati.
- **Strato di output**: Un singolo neurone di output, utilizzato per la classificazione binaria (vittoria o sconfitta del caso).

Usando la funzione introdotta nell'esempio 1 possiamo disegnare la rete MLP che vogliamo adottare per risolvere il problem di classificazione in studio.
Si noti che è necessario eseguire il codice dell'esempio 1 per poter eseguire il seguente codice altrimenti Python segnalerà come errore il fatto di non conoscere la funzione draw_mlp().

In [ ]:
# Parametri della rete MLP utilizzata nell'esempio
hidden_layers = (10, 5)  # Due strati nascosti con 10 e 5 neuroni rispettivamente
input_size = 3  # Tre caratteristiche in input
output_size = 1  # Un neurone di output (classificazione binaria)

# Disegnare la rappresentazione della rete MLP
draw_mlp(hidden_layers, input_size, output_size)

L'implementazione in Python della rete MLP per il nostro problema di classificazione è la seguente:

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.neural_network import MLPClassifier
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split
from sklearn.metrics import ConfusionMatrixDisplay, classification_report
from sklearn.preprocessing import StandardScaler # Aggiunto import

# Simuliamo un dataset per predire se un caso giudiziario sarà vinto o perso basandosi su tre caratteristiche
# Ad esempio, complessità del caso, esperienza dell'avvocato, e importanza mediatica

# Generare dati di esempio
X, y = make_classification(n_samples=200, n_features=3, n_informative=3, n_redundant=0, n_clusters_per_class=1, random_state=42)

# Dividere i dati in train e test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# --> Aggiunta sezione per lo scaling
# Scalare i dati (buona pratica per MLP)
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)
# <-- Fine sezione scaling

# Creare e addestrare un modello MLP usando i dati scalati
mlp_model = MLPClassifier(hidden_layer_sizes=(10, 5), max_iter=1000, random_state=42)
mlp_model.fit(X_train_scaled, y_train) # Usa X_train_scaled

# Predire sul set di test scalato
y_pred = mlp_model.predict(X_test_scaled) # Usa X_test_scaled

# Mostrare la matrice di confusione usando i dati scalati
ConfusionMatrixDisplay.from_estimator(mlp_model, X_test_scaled, y_test, display_labels=["Perso", "Vinto"], cmap=plt.cm.Blues) # Usa X_test_scaled
plt.title('Matrice di Confusione')
plt.show()

# Visualizzare il rapporto di classificazione
report = classification_report(y_test, y_pred, target_names=["Perso", "Vinto"])
print(report)

**Analisi dei Risultati**

1. **Matrice di Confusione**: La matrice di confusione mostra le prestazioni del modello nella classificazione dei casi giudiziari come "Vinto" o "Perso". Nel set di test, il modello ha classificato correttamente la maggior parte dei casi, con solo pochi errori. La matrice di confusione indica che il modello ha identificato con una buona precisione sia i casi vinti che quelli persi.

2. **Rapporto di Classificazione**: 
   - **Precisione**: La precisione per i casi persi è del 97%, mentre per i casi vinti è dell'100%. Questo significa che quando il modello prevede un caso come "Perso", nel 97% dei casi ha ragione, mentre per i casi "Vinto", la precisione è del 100%.
   - **Recall**: La recall per i casi persi è del 100% e per i casi vinti è del 96%. Questo indica che il modello è riuscito a identificare correttamente la quasi totalità dei casi vinti e persi.
   - **F1-score**: L'F1-score, che rappresenta un bilanciamento tra precisione e recall, è del 99% per i casi persi e del 98% per i casi vinti, riflettendo una ottima performance complessiva del modello.

**Osservazioni**: 
   - Il modello ha raggiunto un'accuratezza complessiva del 98%, che è un buon risultato considerando che i dati generati non sono perfettamente separabili.
   - È importante notare che il modello non ha raggiunto il valore di convergenza entro il numero massimo di iterazioni impostato (1000), come indicato dall'avviso di convergenza. Questo suggerisce che con ulteriori iterazioni o con l'ottimizzazione dei parametri del modello, le prestazioni potrebbero migliorare ulteriormente.
   - In sintesi, l'MLP si è dimostrato efficace nel classificare correttamente i casi giudiziari in base alle caratteristiche fornite, anche in presenza di dati non perfettamente distinti. 
   - Questo esempio mostra il potenziale delle reti neurali multistrato per applicazioni giuridiche, come la predizione degli esiti legali, pur sottolineando l'importanza di una corretta configurazione e addestramento del modello per ottenere i migliori risultati possibili.